In [ ]:
!pip install ultralytics==8.0.20
import ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/yolo/yolov8

/content/gdrive/MyDrive/yolo/yolov8


In [ ]:
!yolo task=detect mode=train model=yolov8m.pt data=custom_data.yaml epochs=25 imgsz=800 plots=True

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=custom_data.yaml, epochs=25, patience=50, batch=16, imgsz=800, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, wo

In [ ]:
!yolo task=detect mode=val model=runs/detect/train10/weights/best.pt data=custom_data.yaml

2024-01-06 02:28:46.491799: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 02:28:46.491859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 02:28:46.493332: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 02:28:47.591823: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/gdrive/MyDrive/yolo/yolov8/data/val/labels.cache..

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train10/weights/best.pt conf=0.3 source=im4 save=True imgsz=8688

2024-01-06 20:05:47.229947: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 20:05:47.230005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 20:05:47.231291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 20:05:48.321900: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
WARNING ⚠️ --img-size [8688] must be multiple of max stride 32, updating 

In [ ]:
model = YOLO('runs/detect/train8/weights/best.pt')
results = model.predict(source='ID7685_Arctiidae_Zygaenidea', conf=0.3, imgsz=1920)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


In [ ]:
filenameimg = 'ID7685_Arctiidae_Zygaenidea'
import os
#coordonnées de la zone de comptage
pyi = 255
pyf = 825
pxi = 500
pxf = 1380


#nb d'images en x et en y
if len(os.listdir(filenameimg))>36:
  if len(os.listdir(filenameimg))>42:
    ny = 12
  else:
    ny = 7
else:
    ny = 6

nx = 6
count_part = 0
count_nb = 0
tab = []
for i in range(len(os.listdir(filenameimg))):
  xx = (count_part//ny)
  yy = (count_part%ny)



  pxii = pxi
  pyii = pyi
  pxff = pxf
  pyff = pyf

  #augmentation de la taille de la zone de comptage s'il s'agit d'un bord pour le prendre en compte
  if(xx == 0):
      pxii = 0
  if(xx == nx-1):
      pxff = 1920
  if(yy == 0):
      pyii = 0
  if(yy == ny-1):
      pyff = 1080
  counti = 0
  for j in range(len(results[i].boxes.xyxy)):

    xi = int(results[i][j].boxes.xyxy[0][0].item())
    yi = int(results[i][j].boxes.xyxy[0][1].item())
    if xi > pxii and xi < pxff and yi > pyii and yi < pyff:
      count_nb += 1
      counti +=1

  tab.append(counti)
  count_part +=1
print(count_nb)

149
72
[2, 4, 3, 5, 1, 1, 1, 0, 2, 2, 3, 2, 1, 1, 3, 3, 2, 3, 0, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 1, 4, 2, 2, 3, 0, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 5, 3, 2, 3, 1, 1, 0, 2, 1, 3, 3, 3, 2, 3, 3, 5, 0, 1, 1, 1]


In [ ]:
tab2 = []

for i in range (len(results)):
  tab2.append(len(results[i]))

print(tab2)

[44, 121, 65, 77, 133, 233, 269, 300, 24, 61, 40, 16, 31, 120, 200, 158, 218, 40, 33, 282, 231, 123, 83, 27, 16, 55, 58, 300, 102, 42]
